<a href="https://colab.research.google.com/github/jluisespejo012-pixel/evaluacion-formativa-2/blob/main/Copia_JLE_de_Cart_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 31005_A2: Programming the decision tree (CART) algorithm

Hong Kung(12431868) & Weixiang Gao(12765653)

Video link: https://www.youtube.com/watch?v=koAg_y_l3VY&feature=youtu.be
<br>

## Introduction



El análisis de datos mediante la construcción de modelos se ha utilizado ampliamente
en diversos campos, especialmente para analizar los distintos aspectos de los
productos en el mercado. El diamante es uno de los commodities de inversión más
populares en los últimos años, y su rango de precios es muy amplio debido a las
diferencias de calidad en muchos aspectos.
Por lo tanto, el objetivo de nuestro proyecto es simple: programar un árbol de
decisión para clasificar el rango de precios de los diamantes (bajo, medio, alto). Para
lograr este objetivo, utilizamos el conjunto de datos (diamonds.csv) de Kaggle para
entrenar y probar nuestro modelo de árbol de decisión. La elección más adecuada en
este caso es un árbol binario de estructura simple. Optamos por usar el Árbol de
Clasificación y Regresión (CART, por sus siglas en inglés). Dado que todos los datos
que usamos son discretos, se creará un árbol de clasificación en lugar de un árbol
de regresión. Los detalles del algoritmo del árbol y la construcción del modelo se
discutirán en la siguiente sección.
La entrada de nuestro proyecto es el conjunto de datos de diamantes que se utiliza
para el entrenamiento y la prueba. La salida de nuestro proyecto es el árbol de
decisión CART y su precisión en las pruebas. Finalmente, hablaremos sobre los
problemas éticos relacionados con nuestro proyecto.

## Exploration

### Identify Challenge

The main challenge of this project is CART only support binary split, which means data can only split into two parts(yes answer to splitter, no answer to splitter), the main challenge is to design the model of these issues and determine the splitters under basic CART theory.


### Preparation

To start our program, we need to import all the libraries we need for this project.

In [ ]:
# ndarray processing
import numpy as np
# Dataframe processing
import pandas as pd

# for randomly selecting train/test data from dataframe
import random
# "pretty-print" arbitrary data structure(dict object of a tree in our case) in clearer form
from pprint import pprint

then, let's load and check the source dataframe.

In [ ]:
url = 'https://raw.githubusercontent.com/STKKKKK/UTS_ML_2019_ID12431868/master/31005_ML_A2/diamonds.csv'

df = pd.read_csv(url)
df.head()

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


The dataframe contains the following attributes :<br>
<br>
price : price in US dollars (\$326--\$18,823)

carat : weight in carat (0.2--5.01)

cut : quality of the cut (Fair, Good, Very Good, Premium, Ideal)

color : diamond colour, from J (worst) to D (best)

clarity : the measurement of how clear the diamond is : I1 (worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (best)

x : length in mm (0--10.74)

y : width in mm (0--58.9)

z : depth in mm (0--31.8)

depth : the total depth percentage = z / mean(x, y) = 2 * z / (x + y) (43--79)

table : width of top of diamond relative to widest point (43--95)

In [ ]:
#  a overview

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  53940 non-null  int64  
 1   carat       53940 non-null  float64
 2   cut         53940 non-null  object 
 3   color       53940 non-null  object 
 4   clarity     53940 non-null  object 
 5   depth       53940 non-null  float64
 6   table       53940 non-null  float64
 7   price       53940 non-null  int64  
 8   x           53940 non-null  float64
 9   y           53940 non-null  float64
 10  z           53940 non-null  float64
dtypes: float64(6), int64(2), object(3)
memory usage: 4.5+ MB


And fortunately, there is no null values in the dateset.

From now on, we need to decide which attributes are more 'meaningful' in this project. According to the official diamond grading standard (GIA), the 4 most important factors (4C) are: Carat weight, Cut, Color and Clarity. <br>
Therefore, we will keep these attributes and remove other columns from dataframe.

In [ ]:
df = df.drop(['Unnamed: 0','depth','table','x','y','z'],axis=1)

In addtion, we can easily convert the catagorical data to ***some numerical values*** since these data are 'scales' of a diamond. According to diamond GIA, we use the whole grading scale.

In [ ]:
grade = {'cut': ['Fair','Good','Very Good','Premium','Ideal'],
         'color': ['J','I','H','G','F','E','D'],
         'clarity': ['I3','I2','I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF','FL']}
         # ordered from worst to best

for key, scale in grade.items():
   for index in range(len(scale)):
      df[key] = df[key].map(lambda x: index if x == grade[key][index] else x)

Ofcourse, as we mention we also need to covert the price in US dollars into the low, medium and high grade. Before that, we need to roughly take a overview of price value distrubution.

In [ ]:
df.describe()

,carat,cut,color,clarity,price
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,0.797940,2.904097,3.405803,5.051020,3932.799722
std,0.474011,1.116600,1.701105,1.647136,3989.439738
min,0.200000,0.000000,0.000000,2.000000,326.000000
25%,0.400000,2.000000,2.000000,4.000000,950.000000
50%,0.700000,3.000000,3.000000,5.000000,2401.000000
75%,1.040000,4.000000,5.000000,6.000000,5324.250000
max,5.010000,4.000000,6.000000,9.000000,18823.000000


In this case, we take value of 1200 and 4000 as the price class border value.

In [ ]:
df['price'] = df['price'].map(lambda x: 'low' if x<1200 else
                              ('high' if x>4000 else 'medium'))

Now, let's finally check the data that only used for this program.

In [ ]:
df.head()

,carat,cut,color,clarity,price
0,0.23,4,5,3,low
1,0.21,3,5,4,low
2,0.23,1,5,6,low
3,0.29,3,1,5,low
4,0.31,1,0,3,low


Moreover, we have to randomly decide the indices of Train and Test dataframe, we will get 200 samples for each.

In [ ]:
random.seed(1)

indices = df.index.tolist()
test_indices = random.sample(population=indices, k=200)

test_df = df.loc[test_indices]
train_df = df.drop(test_indices)

Finally, we will convert train dataframe to numpy 2D array for more convenient processing and assign it to variable 'data'

In [ ]:
data = train_df.values

Note that, we should have these variables that we would use a lot in our program :<br>
- df
- test_df
- train_df
- data

### Modelling

 ![flowchart](https://raw.githubusercontent.com/STKKKKK/UTS_ML_2019_ID12431868/master/31005_ML_A2/A2_flowChart.png)

According to the flow chart, it is shown that a Loop of data splitting need to be achieved. Therefore, we will choose a decision tree (CART) to handle this out. Ofcourse, the ID3 decision tree and C4.5 decison tree may also be alternative approach.  <br>
The key concept of CART decision tree is to indentify the split based on Gini-Score, which is value that descibe the un-purity of a data set. The lower Gini-Score of a data set, the more likely of the data is in same class.<br>
In this case, we need to find out the best split among all potential splits by calculating the lowest Gini-gain of these splits.<br>
As we also found that the term ***'split' or 'splitter'*** is mentioned a lot. It is a good way to define it into an abstract object !

## Methodology

### a SPLIT object

In [ ]:
class Split():

    def __init__(self, data, column, value):
        """
        column: SPLIT attribute (column index of data)
        value: SPLIT value
        data_below/above: data set which sperated by this SPLIT
        """
        self.column = column
        self.value = value
        self.data_below = data[data[:, self.column] < value]
        self.data_above = data[data[:, self.column] >= value]


    def formatt(self):
        """
        Represent the SPLIT object in a more intuitive way
        """
        return "{} < {}".format(df.columns[self.column], self.value)


    def gini_gain(self):
        """
        Calculate the Gini-gain of the SPLIT
          consider this value uses both sides data sets' Gini-Score
        """
        def gini(data):
            """
            Calculate the Gini-Score of a data set
            """
                # get the count of unique price grade
            _, counts = np.unique(data[:, -1], return_counts=True)
            p = counts / counts.sum()

                # Formula of Gini-Score:
            Gini = 1 - sum(p ** 2)

            return Gini

        n_data = len(self.data_below) + len(self.data_above)
        p_below = len(self.data_below) / n_data
        p_above = len(self.data_above) / n_data

            # Formula of Gini-gain:
        Gini_gain = p_below*gini(self.data_below)+ p_above*gini(self.data_above)

        return Gini_gain

In [ ]:
# small test

Split(data, 3, 1.8).gini_gain()

np.float64(0.6655809650983266)

### Get Best Split

In [ ]:
def get_best_split(data):
    """
    1. Initially, we need to get all the potential splits of the data set.
    This can be done by finding ALL the Different data points in the
    data set, and the Split will take the avarage value of each two points
    in order to let both points settled away from the Split as possible.

    2. After that, we will find the best Split based on its gini_gain.
    """
    potential_splits = []
    _, n_columns = data.shape

    for column in range(n_columns -1):   # we don't need the price column
        unique_values = np.unique(data[:, column])

        for index in range(len(unique_values)):
                # The first value won't be processed
                # because it doesn't have a previous value
            if index > 0:
                current = unique_values[index]
                previous = unique_values[index -1]
                split = Split(data, column, (current + previous)/2)
                potential_splits.append(split)


       # iterate from all potential splits
       # a gini_gain value will never greater than 1, so we can start from here
    max_gini = 1

    for split in potential_splits:
          if split.gini_gain() <= max_gini:
            max_gini = split.gini_gain()
            best_split = split

    return best_split

In [ ]:
# small test: this is the first split to data

sp = get_best_split(data)
sp.formatt()

'carat < 0.495'

### The Decision Tree

In [ ]:
def train(data, max_depth, depth = 0):
    """
    This function aimed to build the CART decision tree by training the data.
    This is a Recursive Function!
    max_depth: can be set to define the maximum depth of tree

    The tree will be a nested dict object, consider it is built up by sub trees
     with three nodes: {root: [true_leaf, false_leaf]}
    """
    price_column = data[:, -1]
    unique_classes, count = np.unique(price_column, return_counts=True)

        # if price grade unique, or the reach the set max_depth
    if(len(unique_classes) == 1) or (depth == max_depth):
           # classify
        end_node = unique_classes[count.argmax()]

        return end_node

    else:
        depth += 1    # count until reach the max_depth

        best_split = get_best_split(data)

           # buliding tree
        root = best_split.formatt()
        sub_tree = {root: []}
        true_leaf = train(best_split.data_below, max_depth, depth)
        false_leaf = train(best_split.data_above, max_depth, depth)

           # make sure that same leaf won't represent at same time
        if true_leaf != false_leaf:
            sub_tree[root].append(true_leaf)
            sub_tree[root].append(false_leaf)
        else: sub_tree = true_leaf

        return sub_tree

In [ ]:
def test(data, tree):
    """
    The function aimed to test the data by using a CART decision tree.
    This is a Recursive Function!
    """
    root = list(tree.keys())[0]    # get the root node of sub_tree
    column, less_than, value = root.split()

    if data[column] < float(value):   # 'value' needs to convert from str
        leaf = tree[root][0]
    else: leaf = tree[root][1]

    if type(leaf) == dict:    # still haven't reach the end node of whole tree
        return test(data, leaf)
    else: return leaf     # finally reach the end node

## Evaluation

### Performance

**First, let's do a simple test to check if the tree testing function works:**

In [ ]:
tree = train(data, 5)
print(test_df.iloc[14],'\n')
print("The price is '{}'".format(test(test_df.iloc[14], tree)))

carat      1.03
cut           4
color         5
clarity       4
price      high
Name: 13759, dtype: object 



NameError: name 'tree' is not defined

The test seems work and we can find out that the 14th test (out of 200 samples) is a correct one.
<br>
<br>



**Now, we can Train the data and Create a decision tree with depth of 5.**

In [ ]:
tree = train(data, 5)

pprint(tree)  # use 'pretty-print'

**After that, we will apply the test samples to do the test. And finally, we can calculate the accuracy.**

In [ ]:
def accuracy(data, tree):

     # create a new column of test results
  test_df['test_result'] = test_df.apply(test, axis=1, args=(tree,))

     # calculate the mean of boolean values of correctness
     #  (1 if correct, 0 if wrong) , we get the Accuracy of the 'tree'
  accuracy = (test_df.test_result == test_df.price).mean()

  print(test_df.head())
  print('\n','\n','The Accuracy of our CART decision tree is {}'.format(accuracy))


In [ ]:
accuracy(test_df, tree)

<br>
<br>

**Let's Train and Test again with a tree of depth of 6.**

In [ ]:
tree_depth_6 = train(data, 6)
accuracy(test_df, tree_depth_6)

### Effiency

In [ ]:
%timeit tree_depth_of_4 = train(data, 4)

In [ ]:
%timeit tree_depth_of_5 = train(data, 5)

In [ ]:
%timeit tree_depth_of_6 = train(data, 6)

The overall time effiency is not as satisfied as accuracy, that's why we always need to limit the depth of the tree according to different situations. (bigger/deeper the tree, higher the accuracy but lower the effiency)

### Comparative Study

The main concept of CART algorithm is Gini-gain. In fact, there are two others types of widely-used decision tree that use the different concept and have its own limitation:
- ID3 decision tree is a greedy algorithm, decision making is based on information gain, by finding the split of the lowest overall entropy(a value descibe chaostic of information), which means the highest information gain.
 However ID3 decision tree can not deal with continous data.
- C4.5 decision tree is an advanced version of ID3, instead of finding information gain, it turned to find out the information gain ratio.
 C4.5 algorithm support both continous data and discrete data just like CART, but it still tend to fit discrete data better.
- CART is what we are programming in this project. The most significant limitation of CART is binary splis, this may lead to the data over-fitting issue.

We can also use the sklearn library to implement all decision tree algorithms. Note that we can also use the iris data to do the training and testing.

In [ ]:
from sklearn.datasets import load_iris
from sklearn import tree
import graphviz

iris = load_iris()
X = iris.data
y = iris.target
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X,y)

 # dot data graph
data = tree.export_graphviz(clf, out_file=None,
                                 feature_names=iris.feature_names,
                                 class_names=iris.target_names,
                                 filled=True,
                                 rounded=True,
                                 special_characters=True)

graph = graphviz.Source(data)
graph.view()

## Conclusion

To conclude, within the limitation of time and knowledge, we learned how to overcome the difficulties during developing and felt very accomplished when the model built successfully and could achieve the goal of the project, it finds the price rank of a diamond mainly influenced by analysing the carat weight, cut, colour and clarity of a diamond. The accuracy is acceptable enough (about 0.95 when the tree depth is 5). Ofcourse, we can also figure how to build a regression CART and there are several improvements of current CART algorithm, such as how to handle with data over-fitting, as these require much deeper understanding of tree model structuring and the whole machine learning field.   

## Ethical

Ethical considerations are inevitable as computers are playing more and more important roles in humans’ life. There are different positions on ethics with the machine. Some might argue that machine ethics exists because humans are machines and humans have ethics. Others could argue that machine ethics doesn’t exist because ethics is simply emotional expression and machines can’t have emotions (Moor, 2006). Although the discussion of this may quickly bring us into philosophical issues, we cannot ignore it. we can’t—and shouldn’t—avoid consideration of machine ethics in today’s technological world, said by Moor, (2006). According to Kantian duty-based approach, it is not things that affect people, but people that influence things. It is our people who are constructing the real world. In the process of knowing things, people are more important than things themselves. People are morally autonomous. Although human behaviour is restricted by objective causality, people become human beings because they have moral freedom, can transcend cause and effect, and can be responsible for their actions. Therefore, it is important to determine the correct ethics, and ethics must have supreme authority. Ethics measure whether the action itself is in line with ethical standards, without it as a result of happiness. Considering that our project is the model to analyse the determining factors of the price of a diamond, it is a guide for the customers who are not professional in this field. It could help them to identify whether the diamonds they buy are worth the corresponding price. The market is determined by the relationship between supply and demand so our model will not destroy the balance. Within our knowledge, our project is unlikely to be misused.

## Video Pitch


https://www.youtube.com/watch?v=koAg_y_l3VY&feature=youtu.be

## Reference

LUMERA: The 4 C's Of Diamonds, & So Much More https://www.lumeradiamonds.com/diamond-education/index <br>
Moor, J. (2006). The Nature, Importance, and Difficulty of Machine Ethics. IEEE Intelligent Systems, 21(4), pp.18-21.<br>

COMENTARIO PERSONAL

***Idea inicial***: Mi propuesta es utilizar un modelo CART para apoyar el control de calidad en las granulometrias de grueso  de capas del depósito de relaves, permitiendo una evaluación más rápida y objetiva. La idea es entrenar un árbol de decisión capaz de clasificar automáticamente si cada capa cumple con los parámetros establecidos de la fajas granulometrícas determinados en
laboratorio,reduciendo significativamneten errores humanos y fortaleciendo la
consistencia del proceso constructivo.
***Cómo lo ejecutaría***: Reuniría los datos históricos de muestras de suelos ya ejecutados y sus resultados de laboratorio, luego entrenaría un árbol de decisión
que aprenda qué combinaciones de variables corresponden a capas que cumplen o
no con las especificaciones. Una vez entrenado, se podrían ingresar nuevas muestras
y el modelo indicaría de manera rápida y objetiva si cumplen los criterios
establecidos.
***Comentario final**: Con esta herramienta se reducirían los errores humanos, se estandarizaría la evaluación de granuometrias y se agilizaría el control de calidad, contribuyendo a la seguridad y estabilidad del depósito de relaves.